In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from tqdm import tqdm
from keras.applications.vgg16 import VGG16
import cv2
import math
import os
from glob import glob
from scipy import stats as s

from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.applications import imagenet_utils
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
#imports the mobilenet model and discards the last 1000 neuron layer.
base_model = MobileNet(weights='imagenet',include_top=False) 

C:\Users\Jatin_Thakkar\AppData\Local\Continuum\anaconda3\envs\tensorflow_env\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [3]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(50176,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
# loading the trained weights
model.load_weights("weight_MobileNet.hdf5")

In [5]:
# compiling the model
from keras.optimizers import SGD
opt = SGD()
model.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [6]:
# creating the tags
train = pd.read_csv('train_new.csv')
y = train['class']
y = pd.get_dummies(y)

In [7]:
# predictions for test videos
all_videos = pd.read_csv("Test_Data.csv")
test_videos = all_videos['video_name']

In [8]:
def count_frames_manual(video):
	# initialize the total number of frames read
	total = 0

	# loop over the frames of the video
	while True:
		# grab the current frame
		(grabbed, frame) = video.read()
	 
		# check to see if we have reached the end of the
		# video
		if not grabbed:
			break

		# increment the total number of frames read
		total += 1

	# return the total number of frames in the video file
	return total

In [9]:
predict = []
actual = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    count = 0
    videoFile = test_videos[i]
    print(videoFile)
    cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
    frameRate = cap.get(10) #frame rate
    x=1
    total = count_frames_manual(cap)
    start_frame = int(3*total/4)
    # removing all other files from the temp folder
    files = glob('temp/*')
    for f in files:
        os.remove(f)
    #while(cap.isOpened()):
    for i in range(start_frame,total):
        cap.set(1,i)
    #    frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
    #    if (frameId % math.floor(frameRate) == 0):
        # storing the frames in a new folder named train_1
        filename ='temp/' + "frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
    #cap.release()

    # reading all the frames from temp folder
    images = glob("temp/*.jpg")

    prediction_images = []
    for i in range(len(images)):
        img = image.load_img(images[i], target_size=(224,224,3))
        img = image.img_to_array(img)
        img = img/255
        prediction_images.append(img)

    # converting all the frames for a test video into numpy array
    prediction_images = np.array(prediction_images)
    # extracting features using pre-trained model
    prediction_images = base_model.predict(prediction_images)
    # converting features in one dimensional array
    prediction_images = prediction_images.reshape(prediction_images.shape[0], 7*7*1024)
    # predicting tags for each array
    prediction = model.predict_classes(prediction_images)
    # appending the mode of predictions in predict list to assign the tag to the video
    predict.append(y.columns.values[s.mode(prediction)[0][0]])
    # appending the actual tag of the video
    actual.append(videoFile.split('/')[1].split('_')[1])
#    print("Video, Actual, Predicted: ")
#    print(test_videos[i], videoFile.split('/')[1].split('_')[1], y.columns.values[s.mode(prediction)[0][0]])
    

  0%|                                                                                                                                               | 0/44 [00:00<?, ?it/s]

C_Straight/C_Straight_shot161.mp4


  2%|███                                                                                                                                    | 1/44 [00:11<08:23, 11.71s/it]

A_OffSide/A_OffSide_shot14.mp4


  5%|██████▏                                                                                                                                | 2/44 [00:22<07:58, 11.40s/it]

...

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉      | 42/44 [03:48<00:10,  5.43s/it]

A_OffSide/A_OffSide_shot165.mp4


 98%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 43/44 [03:56<00:06,  6.15s/it]

A_OffSide/A_OffSide_shot23.mp4


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [03:59<00:00,  5.45s/it]


In [10]:
# checking the accuracy of the predicted tags
from sklearn.metrics import accuracy_score
accuracy_score(predict, actual)*100

79.54545454545455

In [11]:
from sklearn.metrics import confusion_matrix

In [12]:
cnf_matrix = confusion_matrix(actual, predict)
cnf_matrix

array([[ 6,  3,  0],
       [ 0, 28,  0],
       [ 1,  5,  1]], dtype=int64)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(actual, predict, labels=['Legside', 'OffSide', 'Straight']))

              precision    recall  f1-score   support

     Legside       0.86      0.67      0.75         9
     OffSide       0.78      1.00      0.88        28
    Straight       1.00      0.14      0.25         7

    accuracy                           0.80        44
   macro avg       0.88      0.60      0.62        44
weighted avg       0.83      0.80      0.75        44



In [13]:
actual_video = []

# for loop to extract frames from each test video
for i in tqdm(range(test_videos.shape[0])):
    videoFile = test_videos[i]
    actual_video.append(videoFile)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 17803.34it/s]


In [14]:
test_results = pd.DataFrame({'predict':predict, 'actual':actual, 'video':actual_video})
test_results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
predict    44 non-null object
actual     44 non-null object
video      44 non-null object
dtypes: object(3)
memory usage: 1.2+ KB


In [15]:
test_results['Flag'] = (test_results['predict']==test_results['actual']).astype(int)

In [16]:
test_results

,predict,actual,video,Flag
0,OffSide,Straight,C_Straight/C_Straight_shot161.mp4,0
1,OffSide,OffSide,A_OffSide/A_OffSide_shot14.mp4,1
2,OffSide,Straight,C_Straight/C_Straight_shot21.mp4,0
3,OffSide,OffSide,A_OffSide/A_OffSide_shot155.mp4,1
4,OffSide,OffSide,A_OffSide/A_OffSide_shot71.mp4,1
5,Legside,Legside,B_Legside/B_Legside_shot171.mp4,1
6,OffSide,OffSide,A_OffSide/A_OffSide_shot121.mp4,1
7,OffSide,OffSide,A_OffSide/A_OffSide_shot145.mp4,1
8,OffSide,Straight,C_Straight/C_Straight_shot63.mp4,0
9,Legside,Straight,C_Straight/C_Straight_shot66.mp4,0
